# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

Wykonaj zapytania przy użyciu DBMS:  
  
# 1 Znajdź listę wszystkich filmów o tej samej długości.

In [1]:
import psycopg2 as pg
import pandas as pd
import main

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

df = pd.read_sql('select title, length from film', con=connection).pivot(index=['length', 'title'], columns=[]).sort_index()
df

C:\Users\Mithrandir\AppData\Local\Temp\ipykernel_9252\3361070778.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select title, length from film', con=connection).pivot(index=['length', 'title'], columns=[]).sort_index()


Empty DataFrame
Columns: []
Index: [(46, Alien Center), (46, Iron Moon), (46, Kwai Homeward), (46, Labyrinth League), (46, Ridgemont Submarine), (47, Divorce Shining), (47, Downhill Enough), (47, Halloween Nuts), (47, Hanover Galaxy), (47, Hawk Chill), (47, Shanghai Tycoon), (47, Suspects Quills), (48, Ace Goldfinger), (48, Heaven Freedom), (48, Midsummer Groundhog), (48, Notting Speakeasy), (48, Odds Boogie), (48, Paradise Sabrina), (48, Pelican Comforts), (48, Rush Goodfellas), (48, Stepmom Dream), (48, Sunset Racer), (48, Valentine Vanishing), (49, Doors President), (49, Grosse Wonderful), (49, Heavenly Gun), (49, Hook Chariots), (49, Hurricane Affair), (50, Adaptation Holes), (50, Blues Instinct), (50, Crossing Divorce), (50, Lion Uncut), (50, Muppet Mile), (50, Natural Stock), (50, Pilot Hoosiers), (50, Smoking Barbarella), (50, Zorro Ark), (51, Champion Flatliners), (51, Deep Crusade), (51, English Bulworth), (51, Excitement Eve), (51, Frisco Forrest), (51, Hall Cassidy), (51, Simon North), (52, Caddyshack Jedi), (52, Harper Dying), (52, Lust Lock), (52, Side Ark), (52, Spartacus Cheaper), (52, Trojan Tomorrow), (52, Westward Seabiscuit), (53, Beneath Rush), (53, Cabin Flash), (53, Gump Date), (53, Magnificent Chitty), (53, Movie Shakespeare), (53, Primary Glass), (53, Summer Scarface), (53, Tequila Past), (53, Thin Sagebrush), (54, Airport Pollock), (54, Go Purple), (54, Juggler Hardly), (54, Kill Brotherhood), (54, October Submarine), (54, Sense Greek), (55, Coast Rainbow), (55, Wolves Desire), (56, Bride Intrigue), (56, Cupboard Sinners), (56, Destiny Saturday), (56, Goodfellas Salute), (56, Matrix Snowman), (57, Alter Victory), (57, Cranes Reservoir), (57, Dawn Pond), (57, Doctor Grail), (57, Mosquito Armageddon), (57, Noon Papi), (57, Storm Happiness), (58, Closer Bang), (58, Dances None), (58, Fantasy Troopers), (58, Jekyll Frogmen), (58, Oklahoma Jumanji), (58, Rings Heartbreakers), (58, Super Wyoming), (59, Commandments Express), (59, Daughter Madigan), (59, Heartbreakers Bright), (59, Interview Liaisons), (59, Jet Neighbors), (59, Legend Jedi), (59, Minority Kiss), (59, Vision Torque), (59, Whisperer Giant), (60, Bubble Grosse), (60, Jersey Sassy), (60, Mockingbird Hollywood), (60, Phantom Glory), ...]

[1000 rows x 0 columns]

Odpowiedź: Lista filmów o tej samej długości wyświetlona powyżej.

# 2 Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [25]:
df = pd.read_sql('select c.first_name, c.last_name, c2.city from customer c left join address a on c.address_id = a.address_id '
            'left join city c2 on a.city_id = c2.city_id', con=connection).groupby('city').filter(lambda customers: len(customers) > 1).pivot(index=['city', 'first_name', 'last_name'], columns=[]).sort_index()
df


C:\Users\Mithrandir\AppData\Local\Temp\ipykernel_9252\891258279.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select c.first_name, c.last_name, c2.city from customer c left join address a on c.address_id = a.address_id '


Empty DataFrame
Columns: []
Index: [(Aurora, Clinton, Buford), (Aurora, Scott, Shelley), (London, Cecil, Vines), (London, Mattie, Hoffman)]

# 3 Oblicz średni koszt wypożyczenia wszystkich filmów.

In [3]:
df = pd.read_sql('select film.title, avg(film.rental_rate) from film group by film.title',con=connection)
df

C:\Users\Mithrandir\AppData\Local\Temp\ipykernel_9252\3827652722.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select film.title, avg(film.rental_rate) from film group by film.title',con=connection)


,title,avg
0,Graceland Dynamite,4.99
1,Opus Ice,4.99
2,Braveheart Human,2.99
3,Wonderful Drop,2.99
4,Rush Goodfellas,0.99
...,...,...
995,Gathering Calendar,0.99
996,Drums Dynamite,0.99
997,Samurai Lion,2.99
998,Pond Seattle,2.99


Odpowiedź: Średnie koszty wypożyczenia filmów wypisane powyżej.

# 4 Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [4]:
df = pd.read_sql('select c.name as category,count(f.title) as films_in_category from film f '
                 'left join film_category fc on f.film_id = fc.film_id left join category c ON fc.category_id = c.category_id GROUP BY c.name',con=connection)
df

C:\Users\Mithrandir\AppData\Local\Temp\ipykernel_9252\3719830263.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select c.name as category,count(f.title) as films_in_category from film f '


,category,films_in_category
0,Sports,74
1,Classics,57
2,New,63
3,Family,69
4,Comedy,58
5,Animation,66
6,Travel,57
7,Music,51
8,Drama,62
9,Horror,56


Odpowiedź: Ilość filmów w danej kategorii wyświetlona w tabeli powyżej.

# 5 Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [5]:
df = pd.read_sql('select country.country as country, count(c.customer_id) as customer_count from customer c inner join address a on c.address_id = a.address_id inner join city on city.city_id = a.city_id inner join country on city.country_id = country.country_id group by country.country',con=connection)
df

C:\Users\Mithrandir\AppData\Local\Temp\ipykernel_9252\3431064383.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select country.country as country, count(c.customer_id) as customer_count from customer c inner join address a on c.address_id = a.address_id inner join city on city.city_id = a.city_id inner join country on city.country_id = country.country_id group by country.country',con=connection)


,country,customer_count
0,Bangladesh,3
1,Indonesia,14
2,Venezuela,7
3,Cameroon,2
4,Czech Republic,1
...,...,...
103,Tanzania,3
104,Poland,8
105,Greenland,1
106,French Guiana,1


Odpowiedź: Klienci pogrupowani według kraju.

# 6 Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [6]:
df = pd.read_sql('select * from store where store_id in (select store_id from customer group by store_id having (count(store_id) > 100 and count(store_id) < 300))', con=connection)
df

C:\Users\Mithrandir\AppData\Local\Temp\ipykernel_9252\2739506693.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select * from store where store_id in (select store_id from customer group by store_id having (count(store_id) > 100 and count(store_id) < 300))', con=connection)


,store_id,manager_staff_id,address_id,last_update
0,2,2,2,2006-02-15 09:57:12


Odpowiedź: Informacje o sklepie wyświetlone powyżej.

# 7 Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [7]:
df = pd.read_sql('select r.customer_id from rental r inner join customer c on r.customer_id = c.customer_id right join inventory i on r.inventory_id = i.inventory_id inner join film f on i.film_id = f.film_id group by r.customer_id having sum(f.length) / 60 > 200', con=connection)
df

C:\Users\Mithrandir\AppData\Local\Temp\ipykernel_9252\2540522598.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select r.customer_id from rental r inner join customer c on r.customer_id = c.customer_id right join inventory i on r.inventory_id = i.inventory_id inner join film f on i.film_id = f.film_id group by r.customer_id having sum(f.length) / 60 > 200', con=connection)


,customer_id


Odpowiedź: Nikt nie oglądał filmów ponad 200 godzin

# 8 Oblicz średnią wartość wypożyczenia filmu.

In [8]:
df = pd.read_sql('select avg(rental_rate) as avg_film_rental_rate from film',con=connection)
df

C:\Users\Mithrandir\AppData\Local\Temp\ipykernel_9252\923218766.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select avg(rental_rate) as avg_film_rental_rate from film',con=connection)


,avg_film_rental_rate
0,2.98


Odpowiedź: Średnia wartość wypożyczenia filmu wynosi 2,98.

# 9 Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [9]:
df = pd.read_sql("""select c.name as category, avg(f.length) as avg_length from film f 
                    left join film_category fc on f.film_id = fc.film_id
                    left join category c on fc.category_id = c.category_id
                    group by c.name""",con= connection)
df

C:\Users\Mithrandir\AppData\Local\Temp\ipykernel_9252\2828015426.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""select c.name as category, avg(f.length) as avg_length from film f


,category,avg_length
0,Sports,128.202703
1,Classics,111.666667
2,New,111.126984
3,Family,114.782609
4,Comedy,115.827586
5,Animation,111.015152
6,Travel,113.315789
7,Music,113.647059
8,Drama,120.838710
9,Horror,112.482143


Odpowiedź: Średnia wartość długości filmów wyświetlona powyżej w tabeli.

# 10 Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [22]:
df = pd.read_sql("""select c.category_id, c.name category, max(length(film.title)) film_title_len
                    from category c
                    inner join film_category film_cat using (category_id)
                    inner join film using (film_id)
                    group by c.category_id
                    order by c.category_id asc""", con = connection)          
df

C:\Users\Mithrandir\AppData\Local\Temp\ipykernel_9252\2062006297.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""select c.category_id, c.name category, max(length(film.title)) film_title_len


,category_id,category,film_title_len
0,1,Action,23
1,2,Animation,22
2,3,Children,20
3,4,Classics,23
4,5,Comedy,23
5,6,Documentary,22
6,7,Drama,22
7,8,Family,22
8,9,Foreign,20
9,10,Games,21


Odpowiedź: Najdłuższe filmy w kategorii wyświetlone powyżej

# 11 Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [24]:
df = pd.read_sql("""select c.category_id, c.name category, max(film.length) max_film_len
                    from category c
                    inner join film_category film_cat using (category_id)
                    inner join film using (film_id)
                    group by c.category_id
                    order by c.category_id asc""", con = connection)          
df

C:\Users\Mithrandir\AppData\Local\Temp\ipykernel_9252\723786429.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""select c.category_id, c.name category, max(film.length) film_title_len


,category_id,category,film_title_len
0,1,Action,185
1,2,Animation,185
2,3,Children,178
3,4,Classics,184
4,5,Comedy,185
5,6,Documentary,183
6,7,Drama,181
7,8,Family,184
8,9,Foreign,184
9,10,Games,185


Odpowiedź: Najdłuższe filmy w danych kategoriach wyświetlone powyżej.

# Sprawdzenie funkcji z maina

In [12]:
df = main.film_in_category(6)
df

f:\Pliki_materialy_studia\3_rok\AIBD\adampekala_aibd_local\Lab_f_agregujące_classroom\lab2_join-KosoliC\main.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(request, con=connection)


,title,languge,category
0,Academy Dinosaur,English,Documentary
1,Adaptation Holes,English,Documentary
2,Army Flintstones,English,Documentary
3,Beach Heartbreakers,English,Documentary
4,Bed Highball,English,Documentary
...,...,...,...
63,Wars Pluto,English,Documentary
64,Wedding Apollo,English,Documentary
65,Wife Turn,English,Documentary
66,Wrath Mile,English,Documentary


In [13]:
df = main.number_films_in_category(4)
df

f:\Pliki_materialy_studia\3_rok\AIBD\adampekala_aibd_local\Lab_f_agregujące_classroom\lab2_join-KosoliC\main.py:69: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(request, con=connection)


,category,count
0,Classics,57


In [14]:
df_length = main.number_film_by_length(21,123)
df_length

f:\Pliki_materialy_studia\3_rok\AIBD\adampekala_aibd_local\Lab_f_agregujące_classroom\lab2_join-KosoliC\main.py:103: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(request, con=connection)


,length,count
0,46,5
1,47,7
2,48,11
3,49,5
4,50,9
...,...,...
73,119,6
74,120,9
75,121,8
76,122,11


In [15]:
df = main.client_from_city("Moscow")
df

f:\Pliki_materialy_studia\3_rok\AIBD\adampekala_aibd_local\Lab_f_agregujące_classroom\lab2_join-KosoliC\main.py:131: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(request, con=connection)


,city,first_name,last_name
0,Moscow,Catherine,Campbell


In [16]:
df = main.avg_amount_by_length(87)
df


f:\Pliki_materialy_studia\3_rok\AIBD\adampekala_aibd_local\Lab_f_agregujące_classroom\lab2_join-KosoliC\main.py:160: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(request, con=connection)


,length,avg
0,87,2.685122


In [17]:
df = main.client_by_sum_length(1400)
df

f:\Pliki_materialy_studia\3_rok\AIBD\adampekala_aibd_local\Lab_f_agregujące_classroom\lab2_join-KosoliC\main.py:193: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(request, con=connection)


,first_name,last_name,sum
0,Antonio,Meek,1451
1,Leona,Obrien,1588
2,Katherine,Rivera,1615
3,Tiffany,Jordan,1667
4,Jerome,Kenyon,1679
...,...,...,...
593,Wesley,Bull,4808
594,Clara,Shaw,4808
595,Tammy,Sanders,5065
596,Eleanor,Hunt,5360


In [18]:
df = main.category_statistic_length('Action')
df

f:\Pliki_materialy_studia\3_rok\AIBD\adampekala_aibd_local\Lab_f_agregujące_classroom\lab2_join-KosoliC\main.py:220: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(request, con=connection)


,category,avg,sum,min,max
0,Action,111.609375,7143,47,185
